Importando bibliotecas para o desenvolvimento do projeto

In [42]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, classification_report

from sklearn.impute import SimpleImputer

In [3]:
dados = pd.read_csv('Base_Dados/attrition_Rate_Company.csv', sep=',')

https://www.kaggle.com/code/janietran/xyz-company-attrition-rate#Demographics

In [4]:
dados.head()

,EmployeeID,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,2,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,4,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,5,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   Age                      4410 non-null   int64  
 2   Attrition                4410 non-null   object 
 3   BusinessTravel           4410 non-null   object 
 4   Department               4410 non-null   object 
 5   DistanceFromHome         4410 non-null   int64  
 6   Education                4410 non-null   int64  
 7   EducationField           4410 non-null   object 
 8   EmployeeCount            4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [6]:
colunas_object = dados.select_dtypes(include=['object'])
colunas_object.head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18
0,No,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married,Y
1,Yes,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single,Y
2,No,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married,Y
3,No,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married,Y
4,No,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single,Y


In [16]:
dados['Attrition'] = dados['Attrition'].replace({'Yes': 0, 'No': 1})
dados['BusinessTravel'] = dados['BusinessTravel'].replace({'Non-Travel': 0, 'Travel_Frequently': 1, 'Travel_Rarely': 2})
dados['Department'] = dados['Department'].replace({'Sales': 0, 'Research & Development': 1, 'Human Resources': 2})
dados['EducationField'] = dados['EducationField'].replace({'Technical Degree': 0,'Human Resources': 1,'Life Sciences': 2,'Medical': 3,'Marketing': 4,'Other': 5})
dados['Gender'] = dados['Gender'].replace({'Male': 0, 'Female': 1})
dados['JobRole'] = dados['JobRole'].replace({'Research Director': 0,'Research Scientist': 1,'Sales Executive': 2,'Laboratory Technician': 3,'Healthcare Representative': 4,'Sales Representative': 5,'Manager': 6,'Human Resources': 7,'Manufacturing Director': 8})
dados['MaritalStatus'] = dados['MaritalStatus'].replace({'Married': 0,'Single': 1,'Divorced': 2})
dados['Over18'] = dados['Over18'].replace({'Y': 0,'N': 1})


In [17]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   Age                      4410 non-null   int64  
 2   Attrition                4410 non-null   int64  
 3   BusinessTravel           4410 non-null   int64  
 4   Department               4410 non-null   int64  
 5   DistanceFromHome         4410 non-null   int64  
 6   Education                4410 non-null   int64  
 7   EducationField           4410 non-null   int64  
 8   EmployeeCount            4410 non-null   int64  
 9   Gender                   4410 non-null   int64  
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   int64  
 12  MaritalStatus            4410 non-null   int64  
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [25]:
dados = dados.drop(['Over18','StandardHours'], axis=1)

In [30]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


print(dados.corr(method='pearson').round(4))

                         EmployeeID     Age  Attrition  BusinessTravel  \
EmployeeID                   1.0000  0.0086     0.0047         -0.0026   
Age                          0.0086  1.0000     0.1592          0.0248   
Attrition                    0.0047  0.1592     1.0000         -0.0001   
BusinessTravel              -0.0026  0.0248    -0.0001          1.0000   
Department                   0.0035  0.0108    -0.0482          0.0110   
DistanceFromHome            -0.0011  0.0070     0.0097          0.0246   
Education                   -0.0097 -0.0357     0.0151          0.0028   
EducationField              -0.0039 -0.0317     0.0076         -0.0346   
Gender                       0.0132  0.0394     0.0181          0.0392   
JobLevel                    -0.0033 -0.0029     0.0103          0.0461   
JobRole                      0.0095 -0.0037     0.0694          0.0134   
MaritalStatus               -0.0010 -0.0355    -0.0112         -0.0657   
MonthlyIncome                0.0073 -0

In [31]:
y = dados['Attrition']

Age: 0.1592

TotalWorkingYears: 0.1703

YearsAtCompany: 0.1344

YearsWithCurrManager: 0.1562

EnvironmentSatisfaction: 0.1021

JobSatisfaction: 0.1028

In [32]:
X = dados[['Age', 'TotalWorkingYears', 'YearsAtCompany', 'YearsWithCurrManager', 'EnvironmentSatisfaction', 'JobSatisfaction']]

In [33]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=85)

In [34]:
print(
    X_treino.shape,
    X_teste.shape,
    y_treino.shape,
    y_teste.shape,
    (X_treino.shape[0] + X_teste.shape[0])
)

(3087, 6) (1323, 6) (3087,) (1323,) 4410


Decision Tree Classifier

In [35]:
modelo_arvore = DecisionTreeClassifier(random_state=85)

In [36]:
modelo_arvore.fit(X_treino, y_treino)

DecisionTreeClassifier(random_state=85)

In [37]:
y_pred = modelo_arvore.predict(X_teste)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.907785336356765
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.78      0.63      0.70       223
           1       0.93      0.96      0.95      1100

    accuracy                           0.91      1323
   macro avg       0.85      0.80      0.82      1323
weighted avg       0.90      0.91      0.90      1323



KNeighborsClassifier

In [48]:
imputer = SimpleImputer(strategy='mean')

X_train_imputed = imputer.fit_transform(X_treino)
X_test_imputed = imputer.transform(X_teste)


In [50]:
modelo_vizinhos = KNeighborsClassifier(n_neighbors=5)

In [51]:
modelo_vizinhos.fit(X_train_imputed, y_treino)

KNeighborsClassifier()

In [52]:
y_pred = modelo_vizinhos.predict(X_test_imputed)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.8420256991685563
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.55      0.35      0.43       223
           1       0.88      0.94      0.91      1100

    accuracy                           0.84      1323
   macro avg       0.71      0.65      0.67      1323
weighted avg       0.82      0.84      0.83      1323



Linear SVC

In [53]:
modelo_svc = LinearSVC(max_iter=10000,dual=False,random_state=85)

In [55]:
modelo_svc.fit(X_train_imputed, y_treino)

LinearSVC(dual=False, max_iter=10000, random_state=85)

In [56]:
y_pred = modelo_svc.predict(X_test_imputed)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.8314436885865457
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       223
           1       0.83      1.00      0.91      1100

    accuracy                           0.83      1323
   macro avg       0.42      0.50      0.45      1323
weighted avg       0.69      0.83      0.75      1323



c:\Users\pgdel\Desktop\SENACRS\4º Semestre\Inteligência Artificial\Projetos\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pgdel\Desktop\SENACRS\4º Semestre\Inteligência Artificial\Projetos\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pgdel\Desktop\SENACRS\4º Semestre\Inteligência Artificial\Projetos\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp